# 3.Marketing_Case-ELVT-Deploy

# H2O

In this 3rd and last notebook we will deploy our model into Azure Machine learning service

In [110]:
import sys
import pickle
import os
import json
import pandas as pd
import h2o
#!{sys.executable} -m pip install azureml-sdk --user

In [62]:
TEST_DF_FILE = 'marketing_test.pkl'
MODEL_FOLDER = './marketing_model/'
MODEL = 'ELVTmodel'


MODEL_PATH = MODEL_FOLDER + os.listdir(MODEL_FOLDER)[0] #taking first model from folder

## Load Test data

In [8]:
with open(TEST_DF_FILE, 'rb') as handle:
    test = pickle.load(handle)
test

,cons.conf.idx,cons.price.idx,contact,day_of_week,default,emp.var.rate,euribor3m,housing,loan,marital,...,nr.employed,poutcome,profession,responded,schooling,campaign1,custAge1,pastEmail1,pcontacted_pdays,pContacted_previous
0,-36.4,93.994,telephone,wed,no,1.1,4.859,no,no,married,...,5191,nonexistent,blue-collar,no,basic.6y,2,42,no,True,no


## Verify and load model  

In [120]:
#h2o.init()
# model = h2o.load_model(MODEL_PATH)
# h2o.shutdown()

## Create files for AZML service

In order to make the doployment we need to create a docker image that contains all the logic and environment to run the model. To create the image with h2o it is necesary to provide 2 files 

    1) "score.py", the execution_script to make requests to the model 
    2) "myenv.yml", configuration file for conda environment.

### env.yml

In [121]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - python=3.6
  - pip:
    - colorama ==0.3.8
    - future ==0.15.2
    - tabulate ==0.7.5
    - requests ==2.10
    - h2o
    # Required packages for AzureML execution, history, and data preparation.
    - azureml-sdk
    - azureml-core

Overwriting myenv.yml


### score.py

score.py contains two main functions:
    
    - init(), it will be called the first time the container is launched 
    - run(), it is called every time a request is made

In [122]:
%%writefile score.py

import pickle
import json
import numpy as np
import pandas as pd
from azureml.core.model import Model
import h2o

MODEL = 'model'

def init():
    # read in the model file
    global model
    h2o.no_progress()
    h2o.init()

    model_path = Model.get_model_path(model_name = MODEL)

    # load model
    model = h2o.load_model(model_path)
    print("Model Loaded")
        
def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.read_json(data, orient='records')
        
        data_hex = h2o.H2OFrame(data)

        pred = model.predict(data_hex)
        print(pred)
        
        # Send results
        pred = pred.as_data_frame().values.tolist()
        return json.dumps({"result": pred})

    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


## Register model on Azure

In [72]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.webservice import AciWebservice

print("SDK Version:", azureml.core.VERSION)

SDK Version: 1.0.2


In [73]:
# subscription_id = "<SUBSCRIPTION-ID>"
# resource_group = "aml_rg"
# workspace_name = "aml_workspace"
# workspace_region = 'southcentralus'

# #Only one time: create the AML Workspace
# ws = Workspace.create(name = workspace_name,
#                       subscription_id = subscription_id,
#                       resource_group = resource_group, 
#                       location = workspace_region)
# ws.get_details()
# ws.write_config()

# load workspace configuratio from ./aml_config/config.json file
ws = Workspace.from_config()
print('Name: '+ws.name, 'Resource Group: '+ws.resource_group, 'Location: '+ws.location, sep = '\n')

Found the config file in: C:\JP_NOTEBOOKS\aml_config\config.json
Name: aml_workspace
Resource Group: aml_rg
Location: westcentralus


In [65]:
model = Model.register(model_path = MODEL_PATH,
                       model_name = MODEL,
                       tags = {'framework': 'h2o', 'Department':'Marketing' ,'target': "Responded"},
                       description = "Classification model using h2o for marketing purposes",
                       workspace = ws)
print('Model Registered')

Registering model ELVTmodel
Model Registered


In [66]:
for key,value in ws.models.items():
    print("Name:", key,"\tVersion:", value.version)

Name: ELVTmodel 	Version: 2
Name: model 	Version: 1


## Create Docker image and store it on azure

In [112]:
%%time
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                    runtime = "python",
                                                    conda_file = "myenv.yml",
                                                    tags = {'framework': 'h2o', 'Department':'Marketing' ,'target': "Responded"},
                                                    description = "Classification model using h2o for marketing purposes")

image = ContainerImage.create(name = "elvt.image",
                              # this is the model object
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running.......................
SucceededImage creation operation finished for image elvt.image:7, operation "Succeeded"
Wall time: 2min 43s


## Deploy as web service on Azure Container Instance 

In [92]:

aci_service_name = 'aci-elvt'

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 4, 
                                               tags = {'framework': 'h2o', 'Department':'Marketing' ,'target': "Responded"},
                                               description = "Classification model using h2o for marketing purposes")


In [ ]:
%%time
# create an Azure Container Instance service 

aci_service = AciWebservice.deploy_from_image(deployment_config = aciconfig,
                                       image = image,
                                       name = aci_service_name,
                                       workspace = ws)
aci_service.wait_for_deployment(True)
print(aci_service.state)


In [ ]:
#print(aci_service.get_logs())

In [ ]:
print(aci_service.name)
print(aci_service.state)
print(aci_service.location)
print(aci_service.image_id)
print(aci_service.scoring_uri)
print(aci_service.description)
print(aci_service.tags)

## Testing

In [ ]:
print('web service hosted in ACI:', aci_service.scoring_uri)

In [57]:
test_json = json.dumps({"data": test.to_json(orient='records')})
test_json

'{"data": "[{\\"cons.conf.idx\\":-36.4,\\"cons.price.idx\\":93.994,\\"contact\\":\\"telephone\\",\\"day_of_week\\":\\"wed\\",\\"default\\":\\"no\\",\\"emp.var.rate\\":1.1,\\"euribor3m\\":4.859,\\"housing\\":\\"no\\",\\"loan\\":\\"no\\",\\"marital\\":\\"married\\",\\"month\\":\\"may\\",\\"nr.employed\\":5191,\\"poutcome\\":\\"nonexistent\\",\\"profession\\":\\"blue-collar\\",\\"responded\\":\\"no\\",\\"schooling\\":\\"basic.6y\\",\\"campaign1\\":2,\\"custAge1\\":42,\\"pastEmail1\\":\\"no\\",\\"pcontacted_pdays\\":true,\\"pContacted_previous\\":\\"no\\"}]"}'

In [ ]:
prediction = aci_service.run(input_data = test_json)
print(prediction)